# RTED-TDS Co-Sim

NPCC Case

In [1]:
%run -i 'cosim_import.py'
andes.config_logger(stream_level=20)

0.post4553+g5c8f4a5
2.8.0


`caseH` is the start time [H] of the co-simulation, now 10 and 18 are supported.

In [2]:
caseH = 18
ict = False
ecc = False
agc = False

# --- EV Aggregator ---
sse = ev_ssm(ts=caseH, N=50000, step=1, tp=40,
             lr=0.1, lp=60, seed=2022, name="EVA",
             n_pref=1, is_report=True,
             tt_mean=0.2, tt_var=0.05, tt_lb=0, tt_ub=0.4,
             ict=ict, ecc=ecc, agc=agc)
sse.load_A("Aest.csv")
# historical data
ev_num = pd.read_csv("ev_num.csv")

# --- ANDES case ---
dir_path = os.path.abspath('..')
case_path = '/case/npcc_ev.xlsx'
case = dir_path + case_path
ssa = andes.load(case,
                 setup=True,
                 no_output=True,
                 default_config=False)

# Set output mode as 'manual', turn off TDS progress bar
ssa.TDS.config.save_mode = 'manual'
ssa.TDS.config.no_tqdm = 1

# Set load as constant load.
ssa.PQ.config.p2p = 1
ssa.PQ.config.q2q = 1
ssa.PQ.config.p2z = 0
ssa.PQ.config.q2z = 0
ssa.PQ.pq2z = 0

# Turn on ``numba`` can accelerate TDS.
ssa.config.numba = 1


EVA: ts=18[H], 50000 EVs, Total Q=434.54 MWh
Online 14477, Q=362.27 MWh, SoC=0.7435
Power(MW): Pt=-39.0266, Pc=-39.0266, Pd=0.0
Ctrl: 0=7897; 1=6580; 
EVA: Load A from Aest.csv.
Working directory: "/Users/jinningwang/Documents/work/andes/jwang/notes"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Generated code for <PQ> is stale.
Numerical code generation (rapid incremental mode) started...


Generating code for 1 models on 8 processes.


Saved generated pycode to "/Users/jinningwang/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 1 models in 0.1669 seconds.
Parsing input file "/Users/jinningwang/Documents/work/andes/jwang/case/npcc_ev.xlsx"...
Input file parsed in 0.1132 seconds.
System internal structure set up in 0.0246 seconds.


In [3]:
pq_idx = ssa.PQ.idx.v
pq_p0 = ssa.PQ.p0.v
k = 1.3
ssa.PQ.set(src='p0', idx=pq_idx, attr='v', value=k * pq_p0)

True

In [4]:
ssa.PFlow.run()

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: On
   Sparse solver: KLU
 Solution method: NR method
Numba compilation initiated with caching.
Power flow initialized in 0.0884 seconds.
0: |F(x)| = 5.999835422
1: |F(x)| = 14.84350349
2: |F(x)| = 6.105639424
3: |F(x)| = 19932.3758
4: |F(x)| = 4975.160738
5: |F(x)| = 1307.471695
6: |F(x)| = 25754.58227
7: |F(x)| = 683249.3958
Mismatch increased too fast. Convergence not likely.
Power flow failed after 8 iterations for "/Users/jinningwang/Documents/work/andes/jwang/case/npcc_ev.xlsx".


False

In [5]:
ss_init = ssa.TDS.init()

TypeError: object of type 'NoneType' has no len()

## Load synthetic

Create load data ``d_syn``, which is a ``DataFrame`` that have three columns: ``time``, ``s10``, ``h10``. ``time`` is by seconds, ``s10`` is scalar load.

In [ ]:
%run -i 'cosim_loadsyn.py'

## Setup ``ssp`` and ``ssd``

Convert ``ssa`` to pandapower net ``ssp``, add generator cost

The input cost array follow the matpower/pypower format, now only poly_cost is supported

In [ ]:
%run -i 'cosim_setup_dispatch.py'
dc_comp.round(4)

## Prepare

### Make link table

In [ ]:
%run -i 'cosim_linktable.py'
ssa_key2

## Loop

In the loop, there are mainly X parts:

- interval RTED: run DCOPF (``ssd.mdl``), run ACOPF(``ssp``), 

- interval AGC: do AGC, do dispatch with smooth setpoints

- interval PQ: alter load, run TDS(``ssa.TDS``)

Notes:

- The setpoints for DG are coded but not verified yet.

- After the development of ANDES control room, the dispatch and AGC part can be refactored.

- interface variables: ``DG.pmx``: DPV profile, ``DG.pref0``: setpoints, ``DG.pext0``: AGC

- ACOPF in pandapower considered generator limtis of ramping and SFR reserve

In [ ]:
# Change ``rru``, ``rrd``, ``rsfr``, ``t_total`` for necessary.
t_total = 3600

rru, rrd, rsfr = 0.0, 0.0, 0.05

# Define functions and constants used in loop
%run -i 'cosim_const.py'

Check convergence

In [ ]:
# %run -i 'cosim_bmain.py'

Loop

In [ ]:
%run -i 'cosim_main.py'

In [ ]:
ssa.exit_code

In [ ]:
sse.plot_agc(style='ieee')

In [ ]:
%run -i 'cosim_plot.py'

In [ ]:
# --- save output ---
file_beging = 'cosim_'
sim_name = 'ieee39_s'

cosim_out.to_csv(file_beging + sim_name + '_out.csv', index=False)
sse_out.to_csv(file_beging + sim_name + '_sse.csv', index=False)
bu_df.to_csv(file_beging + sim_name + '_bu.csv', index=False)
bd_df.to_csv(file_beging + sim_name + '_bd.csv', index=False)
pg_df.to_csv(file_beging + sim_name + '_pg.csv', index=False)
agc_mile.to_csv(file_beging + sim_name + '_agcm.csv', index=False)
sfr_res.to_csv(file_beging + sim_name + '_sfr.csv', index=False)
ev_agc.to_csv(file_beging + sim_name + '_evagc.csv', index=False)
ev_soc.iloc[0:800].to_csv(file_beging + sim_name + '_evsoc1.csv', index=False)
ev_soc.iloc[800:].to_csv(file_beging + sim_name + '_evsoc2.csv', index=False)
sse.ev.to_csv(file_beging + sim_name + '_evdata.csv', index=False)

import csv
new_path = open(file_beging + sim_name + '_rted.csv', 'w')
z = csv.writer(new_path)
for new_k, new_v in rted_res.items():
    z.writerow([new_k, new_v])
new_path.close()